In [ ]:
import pandas as pd

project_name = "CANDID_DAC"
# set up the experiment we want to plot
# benchmark = "candid_sigmoid"
# dims = [5, 10]
benchmark = "piecewise_linear"
dims = [2, 5, 10]
n_acts = [3, 5, 10]
reward_shape = 'exponential'
exp_reward = 4.6
importance_base = 0.5
n_act = 3

metric_to_plot = "avg_reward_train_set"
# metric_to_plot = "avg_episodic_reward"
config_path = f"../run_data/{project_name}_configs.csv"
metrics_path = f"../run_data/{project_name}_metrics.csv"

df_configs = pd.read_csv(config_path)
df_train = pd.read_csv(metrics_path)

In [ ]:
# get the configs per benchmark dimensionality
configs = df_configs[(df_configs['benchmark'] == benchmark) &
                     (df_configs['reward_shape'] == reward_shape) &
                     (df_configs['exp_reward'] == exp_reward) &
                     (df_configs['importance_base'] == importance_base) &
                     (df_configs['dim'].isin(dims)) &
                     (df_configs['reverse_agents'] == False)]

configs_dim = configs[(configs['n_act'] == n_act) &
                      (configs['dim'].isin(dims))]

configs_n_act = configs[(configs['dim'] == 5) &
                        (configs['n_act'].isin(n_acts))]

configs_dim = configs_dim.groupby(['run_name', 'dim']).agg(list)['run_id']
configs_n_act = configs_n_act.groupby(['run_name', 'n_act']).agg(list)['run_id']

print(configs_dim.apply(lambda x: len(x)))
print(configs_n_act.apply(lambda x: len(x)))

In [ ]:
from matplotlib import pyplot as plt
import numpy as np
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)
from plotting_helpers import get_best_possible_avg_reward, translate_run_name, METHOD_COLOURS

plt.rcParams.update({
    'font.size': 8,           # Global font size
    'axes.titlesize': 7,      # Title size of individual plots
    'axes.labelsize': 7,      # Label size for x and y labels
    'xtick.labelsize': 6.5,      # Size of x-tick labels
    'ytick.labelsize': 6.5,      # Size of y-tick labels
    'legend.fontsize': 7,      # Size of the legend text
    'figure.titlesize': 12,     # Title size of the entire figure
    'lines.linewidth': 0.75
})
width = 6 # in inches, same as width of the latex template
dpi=100  # set to 100 to get in size of paper not zoomed in
fig, axes = plt.subplots(2, len(dims), figsize=(width, width / 2.5), sharey=True, sharex=True, dpi=dpi, )

df_configs.set_index('run_id', inplace=True)

metric_to_plot = 'avg_reward_test_set'
for i, (dim, n_act) in enumerate(zip(dims, n_acts)):
    dim_ids = configs_dim.xs(dim, level='dim')
    n_act_ids = configs_n_act.xs(n_act, level='n_act')

    # get the first config for the current dimensionality and number of actions
    dim_conf = df_configs.loc[dim_ids[0]].to_dict(orient='records')[0]
    n_act_conf = df_configs.loc[n_act_ids[0]].to_dict(orient='records')[0]
    # print(dim_conf)
    # print(n_act_conf)
    # continue

    for j, (conf, ids) in enumerate(zip([dim_conf, n_act_conf], [dim_ids, n_act_ids])):
        dim = conf['dim']
        n_act = int(conf['n_act'])

        # the upper plot takes in dimensional scaling
        ax = axes[j, i]

        # plot the best possible reward
        optimal_reward_1D = get_best_possible_avg_reward(dim=dim, benchmark=benchmark, reward_shape=reward_shape, c=exp_reward,
                                                        importance_base=importance_base, max_dim=1, n_acts=n_act)
        optimal_reward_acc = get_best_possible_avg_reward(dim=dim, benchmark=benchmark, reward_shape=reward_shape, c=exp_reward,
                                                        importance_base=importance_base, max_dim=dim, n_acts=n_act)

        ax.plot([0, 10000], [optimal_reward_acc, optimal_reward_acc], label='optimal', color='black', linestyle='--')
        ax.plot([0, 10000], [optimal_reward_1D, optimal_reward_1D], label='optimal (1D)', color='grey', linestyle='--')

        # only keep the runs for this dimensionality
        # dim_configs = grouped_configs.xs(dim, level='dim')

        # now plot rewards per approach, which is encoded by run_name
        for run_name, run_ids in ids.items():
            run_name = run_name.rsplit('_', 1)[0]
            label = translate_run_name(run_name)
            color = METHOD_COLOURS[run_name]

            # get the rewards for this run, drop all undefined time steps
            rewards = df_train.loc[df_train['run_id'].isin(run_ids)].groupby('_step')[metric_to_plot].agg(['mean', 'std'])
            rewards = rewards.dropna()
            
            # plot the mean and std
            ax.fill_between(rewards.index / 10, rewards['mean'] - rewards['std'], rewards['mean'] + rewards['std'], alpha=0.5, color=color)
            ax.plot(rewards.index / 10, rewards['mean'], label=label, color=color)

            # make the plot square, regardless of scaling difference in x and y
            ax.set_ylim(0, 10.2)
            ax.set_xlim(0, 10000)
            ax.set_yticks(np.linspace(0, 10, 5))
            ax.set_xticks(np.linspace(0, 10000, 5))
            # ax.set_xticks(np.linspace(0, 10000, 5))
            ax.tick_params(axis='x', pad=-5)

            if j == 0:
                title = f'dim={dim}'
            else:
                title = f'n_act={n_act}'
        ax.set_title(title, y=0.94)

for ax in axes[1]:
        ax.set_xlabel('Episodes')

for ax in axes[:, 0]:
    ax.set_ylabel('Avg. episodic reward')

# set a textfield to the left of the first row, stating that n_acts is fixed to 3
axes[0, 0].text(-0.43, 0.5, 'n_act=3', va='center', rotation='vertical', transform=axes[0, 0].transAxes, fontweight='bold')
axes[1, 0].text(-0.43, 0.5, 'dim=5', va='center', rotation='vertical', transform=axes[1, 0].transAxes, fontweight='bold')
# # set the same y-axis label for all plots
# set a shared legend, centered under the plots, considering that legends is identical for all plots
axes[0, -1].legend(loc='center left', bbox_to_anchor=(1, -0.25), ncol=1)
fig.subplots_adjust(wspace=0.12,)
fig.savefig(f'./paper_plots/pl_dim_n_acts_scaling.png', bbox_inches='tight', dpi=600)

In [ ]:
import matplotlib
print(matplotlib.rcParams['figure.dpi'])